In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import userdata
token =userdata.get('Cryptonite-Token')

In [3]:
!git config --global user.email "idantsri2007@gmail.com"
!git config --global user.name "SILETRO"

In [4]:
!git clone https://{token}@github.com/SILETRO/Cryptonite-RTP-Idant.git
%cd Cryptonite-RTP-Idant

Cloning into 'Cryptonite-RTP-Idant'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 176 (delta 13), reused 13 (delta 8), pack-reused 150 (from 1)
Receiving objects: 100% (176/176), 62.45 MiB | 10.74 MiB/s, done.
Resolving deltas: 100% (59/59), done.
/content/Cryptonite-RTP-Idant


In [6]:
!mkdir Trainee-Task0/

In [7]:
!cp '/content/drive/MyDrive/Colab Notebooks/GenAI.ipynb' Trainee-Task0/

In [8]:
!git add .

In [9]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   Trainee-Task0/GenAI.ipynb



In [10]:
!git commit -m "Added notebook"
!git push -u origin main

[main a4b2cf2] Added notebook
 1 file changed, 1 insertion(+)
 create mode 100644 Trainee-Task0/GenAI.ipynb
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (4/4), 23.67 KiB | 5.92 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/SILETRO/Cryptonite-RTP-Idant.git
   8de9bfc..a4b2cf2  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
import nltk

In [2]:
nltk.download('punkt_tab')
file_path = "/content/AllCombined 2.txt"

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
def load_and_chunk_text(file_path, chunk_size=500, overlap=50):
    with open(file_path, 'r', encoding='utf-8') as f:
        full_text = f.read()

    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    tokens = tokenizer.encode(full_text, add_special_tokens=False)

    chunks = []
    start = 0
    while start < len(tokens):
        end = start + chunk_size
        chunk_tokens = tokens[start:end]

        chunk_text = tokenizer.decode(chunk_tokens)
        chunks.append(chunk_text)

        start += (chunk_size - overlap)

    print(f"Total chunks created: {len(chunks)}")
    return chunks

text_chunks = load_and_chunk_text(file_path)

In [ ]:
import sys
!apt install -y libomp-dev
!{sys.executable} -m pip install faiss-cpu

In [ ]:
from sentence_transformers import SentenceTransformer

import faiss
import numpy as np

def create_vector_store(chunks):
    """
    Embeds text chunks and builds a FAISS index.
    """
    # 1. Initialize the embedding model
    # 'all-MiniLM-L6-v2' is fast, lightweight, and standard for RAG
    embedder = SentenceTransformer('all-MiniLM-L6-v2')

    # 2. Generate embeddings (Vectors)
    # FAISS expects float32 format
    embeddings = embedder.encode(chunks, convert_to_numpy=True)
    embeddings = embeddings.astype('float32')

    # 3. Create FAISS Index
    # Dimension matches the model output (384 for MiniLM)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)

    # 4. Add vectors to index
    index.add(embeddings)

    print(f"Index built with {index.ntotal} vectors.")
    return index, embedder

vector_index, embed_model = create_vector_store(text_chunks)

In [6]:
def retrieve_context(query, index, chunks, model, k=3):
    """
    Searches the FAISS index for the top-k most similar chunks.
    """
    # 1. Embed the query
    query_vector = model.encode([query], convert_to_numpy=True).astype('float32')

    # 2. Search the index
    # D is distances (lower is better for L2), I is indices of the chunks
    distances, indices = index.search(query_vector, k)

    # 3. Fetch the actual text
    retrieved_chunks = [chunks[idx] for idx in indices[0]]

    # 4. Basic validity check
    # If distance is too high, the relevance is low. (Threshold depends on model)
    # L2 distance > 1.5 usually means "not very relevant" for normalized vectors
    if distances[0][0] > 1.5:
        return None

    return retrieved_chunks

# --- USAGE TEST ---
# test_context = retrieve_context("What is the main topic?", vector_index, text_chunks, embed_model)
# print(test_context)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def load_generator_model():
    """
    Loads the TensorFlow version of FLAN-T5-Small.
    """
    model_name = "google/flan-t5-small" # 80M params (Fits constraints)

    print("Loading TensorFlow Model...")
    # TF... classes are specifically for TensorFlow/Keras
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name, from_tf=True)

    return tokenizer, model

# --- USAGE ---
gen_tokenizer, gen_model = load_generator_model()

In [17]:
import re
from nltk.tokenize import sent_tokenize

def post_process_answer(answer_text):
    """
    Cleans the answer: removes parens, merges short sentences, enforces limit.
    """
    # 1. Remove parentheses and content inside them
    cleaned = re.sub(r'\([^)]*\)', '', answer_text)

    # 2. Remove extra spaces created by deletion
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()

    # 3. Split into sentences
    sentences = sent_tokenize(cleaned)

    # 4. Logic: Merge very short sentences (fragment fix)
    final_sentences = []
    buffer_sent = ""

    for sent in sentences:
        if len(sent.split()) < 5: # If sentence is < 5 words
            buffer_sent += " " + sent
        else:
            if buffer_sent:
                final_sentences.append(buffer_sent.strip() + " " + sent)
                buffer_sent = ""
            else:
                final_sentences.append(sent)

    if buffer_sent and final_sentences:
        final_sentences[-1] += " " + buffer_sent
    elif buffer_sent:
        final_sentences.append(buffer_sent)

    # 5. Enforce Maximum 4 sentences
    return " ".join(final_sentences[:4])

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def load_generator_model():
    """
    Loads the PyTorch version of FLAN-T5-Small.
    """
    model_name = "google/flan-t5-small" # 80M params (Fits constraints)

    print("Loading PyTorch Model...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    return tokenizer, model

# --- USAGE ---
gen_tokenizer, gen_model = load_generator_model()

In [36]:
def answer_question(question):
    print(f"\nUser Question: {question}")

    # 1. Retrieve
    context_chunks = retrieve_context(question, vector_index, text_chunks, embed_model)

    if not context_chunks:
        return "Not enough information in the Simple Wikipedia dataset."

    # Join chunks into a single string
    context_text = " ".join(context_chunks)

    # 2. Prompt Engineering (Strict Format)
    input_text = f"""You are an expert assistant. Using ONLY the context below, provide a detailed, complete, and well-explained answer.
                If needed, include multiple paragraphs. Context: {context_text} Question:{question} Answer: """


    # 3. Tokenize Input (Return PyTorch tensors)
    inputs = gen_tokenizer(input_text, return_tensors="pt",max_length=10000, truncation=True)

    # 4. Generate Answer (Deterministic)
    outputs = gen_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=800,
        do_sample=False, # Deterministic (Greedy Search)
        num_beams = 3,
        length_penalty =1.1,
        temperature=0.5  # Zero randomness
    )

    # 5. Decode
    raw_answer = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 6. Post-Process
    final_answer = post_process_answer(raw_answer)

    return final_answer

In [37]:
response = answer_question("What is photosynthesis?")
print(f"Agent Answer: {response}")


User Question: What is photosynthesis?
Agent Answer: oxygen gas diffuses out of the plant as a waste product of photosynthetic reactions, and atp is synthesized from adp and inorganic phosphate. this all happens in the grana of chloroplasts. during this reaction, sugars are built up using carbon dioxide and the products of the light - dependent reactions and various other chemicals found in the plant in the calvin cycle. therefore, the light - independent reaction cannot happen without the light - dependent reaction.


In [39]:
answer_question("What did Edison do?")


User Question: What did Edison do?


'Inventor and entrepreneur, who invented many kinds of inventions and inventions. he developed one of the first practical light bulbs, but contrary to popular belief did not invent the light bulb. he started the general electric company to make some of the things he invented. edison developed one of the first practical light bulbs, but contrary to popular belief did not invent the light bulb.'

In [38]:
answer_question("Who is Donald Trump")


User Question: Who is Donald Trump


'president of the united states in the 2024 presidential election. He was acquitted by the senate in february 2022, making him the third president in american history to be impeached twice. in january 2021, trump controversially made a telephone call with georgia secretary of state brad raffensperger. in the call, he was reported to have tried to change the election results.'